Task1_0820. Python REPL 도구를 사용하는 LangChain 예시를 수행하세요.

In [1]:
import sys
import io


class PythonREPL:
    def run(self, code: str) -> str:
        # 실행할 파이썬 코드의 출력값을 캡처
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        try:
            exec(code)  # 코드 실행
        except Exception as e:
            return str(e)
        finally:
            # 기존 stdout 복원
            sys.stdout = old_stdout

        # 실행 결과 반환
        return new_stdout.getvalue()


# 예시 사용
python_repl = PythonREPL()

# 실행할 코드 입력
code = """
a = 5
b = 10
print(a * b)
"""

# Python 코드 실행
result = python_repl.run(code)
print(result)

50



In [3]:
from openai import OpenAI
import os

MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
import os
from uuid import uuid4

# unique_id를 uuid4()로 생성
unique_id = str(uuid4())

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain Tool Use Quickstart - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [5]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain import hub

llm = ChatOpenAI(model=MODEL)
python_repl = PythonREPL()

@tool
def run_code(code: str) -> str:
    """run a python code and return the output"""
    return python_repl.run(code)

tools = [run_code]
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": """
        a = 5
        b = 10
        print(a * b)
        """
    }
)



> Entering new AgentExecutor chain...

Invoking: `run_code` with `{'code': 'a = 5\nb = 10\nprint(a * b)'}`


50
The result of the multiplication \( a \times b \) where \( a = 5 \) and \( b = 10 \) is \( 50 \).

> Finished chain.


{'input': '\n        a = 5\n        b = 10\n        print(a * b)\n        ',
 'output': 'The result of the multiplication \\( a \\times b \\) where \\( a = 5 \\) and \\( b = 10 \\) is \\( 50 \\).'}